# Optimizing QAOA

by Pantita Palittapongarnpim

Now that we have the classes for QAOA circuits, we need to optimize the circuits.

### Package Versions

QuTiP: 4.5.2

Python: 3.8.6

### Loading Packages

In [1]:
import numpy as np
import qutip as qt

### QAOA Classes

In [4]:
class Graph:
    def __init__(self, total_qubit, J):
        self.total_qubit=total_qubit #integer
        self.J=J #matrix: undirected (symmetric)
        
    def Ham(self):
        Ham_out=0;
        for i in range(self.total_qubit):
            for j in range(i,self.total_qubit):
                opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
                Ham_out=Ham_out+J[i][j]*opr
        return(Ham_out)

In [5]:
class QAOA:
    def __init__(self, gamma, beta, p, graph):
        ## what does graph has to contain:
        # total_qubit -- integer
        # J -- double matrix        
        self.gamma=gamma
        self.beta=beta
        self.p=p
        self.J=graph.J
        self.total_qubit=graph.total_qubit
        
    def Uz(self,i,j,l):
        opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
        Hvar=-1j*gamma[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def Ux(self,i,l):
        opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)])
        Hvar=-1j*beta[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def circuit(self):
        Had=qt.tensor([qt.qip.operations.snot() for k in range(self.total_qubit)])
        qaoa_circ=Had
        for l in range(self.p):
            #operate the Uz gates
            for i in range(self.total_qubit):
                for j in range(i,self.total_qubit):
                    if i!= j:
                        qaoa_circ=self.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(self.total_qubit):
                qaoa_circ=self.Ux(i,l)*qaoa_circ
        return(qaoa_circ)
    
    def state(self):
        gnd=qt.basis(2,0)
        state=qt.tensor([gnd for i in range(self.total_qubit)])
        state=self.circuit()*state
        return(state)
 

### Objective functions

Now that we have the simulation of the QAOA circuit, we need to optimize the parameters, namely gamma and beta array.

But before that, we need to pick the objective function we are going to use.

__Average Energy__

This is the traditional objective function for QAOA

__Gibbs__

This one is proposed by Li et al.

__Average Gibbs__

Actually doesn't have a name, proposed by P'Thip.

In [44]:
def AveEnergy(graph,output):
    #output is the output state from the QAOA class
    return(output.dag()*graph.Ham()*output)

def Gibbs(eta,graph,output):
    f=-np.log(output.dag()*(-eta*graph.Ham()).expm()*output)
    return(f)

def AveGibbs(eta,graph,output):
    f=-np.log(output.dag()*graph.Ham()*(-eta*graph.Ham()).expm()*output)
    return(f)

### Testing out the objective functions

In [17]:
total_qubit=3 #number of qubit
p=1 #number of layers

#declaring QAOA parameters
gamma=np.random.rand(p)
beta=np.random.rand(p)

#defining the problem
J=np.random.rand(total_qubit,total_qubit) #creating the couping parameters,still directional in this instance)
for i in range(total_qubit):
    for j in range(i,total_qubit):
        temp=(J[i][j]+J[j][i])/2.0
        J[i][j]=temp
        J[j][i]=temp
#Now J is undirectional

In [18]:
graph = Graph(total_qubit,J)
qaoa = QAOA(gamma,beta,p,graph)

output=qaoa.state()

In [19]:
#Average Energy#
print(AveEnergy(graph,output))

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.47852975]]


In [41]:
#Gibbs energy
eta=0.3 #how do we tune this again?
print(Gibbs(eta,graph,output))

[[0.01020317-0.j]]


In [56]:
#Average Gibbs
eta=0.1
print(AveGibbs(eta,graph,output))

[[1.91288492-0.j]]


Observations:

* As expected, the Gibbs energy depend on eta, but we don't have a good idea what value to use.
* Average Gibbs turns to complex number with fixed imaginary for certain range of eta. I am not sure yet if this is a bug or a characteristic of the function.